In [41]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Jupyter上にHTMLを表示する
from IPython.display import HTML

# YoutubeAPIの利用
from apiclient.discovery import build
from apiclient.errors import HttpError

# spotifyのAPIを利用
import spotipy

In [4]:
df = pd.read_csv('dataset/streams_1.csv', header=0)
df.head()

,Members,SongName,VideoID,StartTime,TrackID,TrackName,Duration,ArtistID,ArtistName,ArtistURL,ArtistGenres
0,星街すいせい,破滅の純情,rbkjVA6Fjh0,4872,4Rs0bABgDMzAqAAd2P8ArD,Hametsu no Junjo,267400,5U8dcKsZ0mmeiUgbM0jvFC,Sheryl Nome starring May'n,https://i.scdn.co/image/ab67616d0000b27364c32c...,None
1,星街すいせい,コワレヤスキ,rbkjVA6Fjh0,5743,1TizL66jSoN48Hj8airJgr,コワレヤスキ,302453,7gKNDpetmXrzHZuLdo9tv8,Guilty Kiss,https://i.scdn.co/image/ab67616d0000b273def723...,love live
2,"星街すいせい,常闇トワ",ライオン,rbkjVA6Fjh0,6763,1ux8Y96cV5Qs0kdOCq4cZ2,Lion,303657,0JQH8OHvGdooprROP18Wg6,May'n,https://i.scdn.co/image/ab6761610000e5eb0d1c05...,"anime,j-poprock"
3,星街すいせい,天球、彗星は夜を跨いで,FZnG1t34wCs,95,0wTs0OV88H5YZrFX6zN4QJ,天球、彗星は夜を跨いで,256548,726WiFmWkohzodUxK3XjHX,Hoshimachi Suisei,https://i.scdn.co/image/ab6761610000e5ebee9358...,"hololive,japanese vtuber"
4,星街すいせい,Neo stream,FZnG1t34wCs,568,1WZ2n937qxjPuJAHzFGegK,NEO STREAM,270960,4b1IVV9meynYx65WpsxjbT,Walkure,https://i.scdn.co/image/ab67616d0000b273ab4899...,None


* 終了時間の予測時間を追加する
* ジャンルを一覧表示し何かできないか確認する
* Membersを見てコラボ情報を追加する
* Youtube APIで動画タイトルを追加する
* サムネやタイトルを見て以下のデータをアノテーションする
    * アカペラ
    * ライブ
* Spotify APIを使って試しにジャンルを打ってみる
    * 割と真面目にデータを見る必要があるため後回し

In [15]:
# 終了時間の追加
start_time = np.array(df['StartTime'])
duration_ms = np.array(df['Duration'])
end_time = start_time + duration_ms//1000

# データが無いものの数を確認し、これらは後程手動で入力する
nodata_num = duration_ms[duration_ms==0].shape[0]

In [16]:
# コラボ情報の追加
members = np.array(df['Members'])
collabs = np.zeros(df.shape[0], dtype=bool)
for i, member in enumerate(members):
    member_list = member.split(',')
    collabs[i] = len(member_list) > 1

In [20]:
# ジャンル一覧の確認
genres = np.array(df['ArtistGenres'])
genres_1 = []
for i, genre_1 in enumerate(genres):
    genres_2 = genre_1.split(',')
    for genre_2 in genres_2:
        genres_1.append(genre_2)
        
genre_all, genre_count = np.unique(np.array(genres_1), return_counts=True)

# 各ジャンルの数を見ながらどのような分類ができるか確認する
# ボカロとかアニメとかでジャンルが決められそうならやってみる
# 楽曲ごとに重複しているものは注意する
genre_all, genre_count

(array(['None', 'anime', 'hololive', 'j-poprock', 'japanese vtuber',
        'love live', 'macross', 'vocaloid'], dtype='<U15'),
 array([6, 2, 2, 1, 2, 2, 1, 1]))

In [25]:
# Youtube APIの準備

# APIキーをファイルから取得
f = open('../secret/apikey', 'r')
api_key = f.read()
f.close()

# APIキーを用いてリクエスト用のクラスを作成
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=api_key)

In [27]:
video_ids = np.array(df['VideoID'])
unique_video_ids = np.unique(video_ids)
unique_titles = []
for video_id in unique_video_ids:
    try:
        video_detail = youtube.videos().list(
            part = 'snippet,statistics,contentDetails', 
            id = video_id, 
        ).execute()
    except HttpError:
        print('データ参照中にエラーが発生しました')
        break
    
    video_snippet = video_detail['items'][0]['snippet']
    # date = video_snippet['publishedAt']
    unique_title  = video_snippet['title']
    unique_titles.append(unique_title)

In [28]:
video_ids_dict = dict(zip(unique_video_ids, unique_titles))
print(video_ids_dict)

titles = []
for video_id in video_ids:
    titles.append(video_ids_dict[video_id])
titles = np.array(titles)

{'FZnG1t34wCs': '【#星街すいせい3D】全編ライブ！歌って踊る姿を見てください！✨【ホロライブ / 星街すいせい】', 'n0ieG2D_qaU': 'ひたすらアカペラする歌枠！【ホロライブ / 星街すいせい】', 'rbkjVA6Fjh0': 'ホロライブのアイドルが集まってカラオケをしたらこうなる【 #ホロドルカラオケ 】'}


In [38]:
def output_html(video_ids, titles):
    html = '<div style="float:left;">'
    for i, video_id in enumerate(video_ids):
        html += ('<img src="http://img.youtube.com/vi/'+video_id+'/sddefault.jpg "alt="取得できませんでした" width="80"><br>')
        url = "https://www.youtube.com/watch?v="+video_id
        html += ('  <a href="'+url+'">'+url+'</a><br>'+video_id+'<br>') 
        html += (''+titles[i]+'<br>') 
    html += '</div>'
    return html

In [39]:
# 表示しながらライブやアカペラにアノテーションする
HTML(output_html(unique_video_ids, unique_titles))

In [ ]:
aka_ids = []
live_ids = []

In [42]:
# Spotify APIの準備

# APIキーをファイルから取得
f = open('../secret/clientsecret_spotify', 'r')
client_secret = f.read()
f.close()
client_id, client_secret = client_secret.split('\n')[0], client_secret.split('\n')[1]

client_credentials_manager = spotipy.oauth2.SpotifyClientCredentials(client_id, client_secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [45]:
track_ids = np.array(df['TrackID'])
for track_id in track_ids:
    result = spotify.audio_features(track_id)
    print(result)
    break

# いくつかのパラメータごとにtop50 & top-50 を取り出してどういう曲が出てくるか見てみる
# https://gaaaon.jp/blog/spotify

[{'danceability': 0.504, 'energy': 0.829, 'key': 5, 'loudness': -5.248, 'mode': 0, 'speechiness': 0.0621, 'acousticness': 0.0315, 'instrumentalness': 0, 'liveness': 0.295, 'valence': 0.771, 'tempo': 165.904, 'type': 'audio_features', 'id': '4Rs0bABgDMzAqAAd2P8ArD', 'uri': 'spotify:track:4Rs0bABgDMzAqAAd2P8ArD', 'track_href': 'https://api.spotify.com/v1/tracks/4Rs0bABgDMzAqAAd2P8ArD', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4Rs0bABgDMzAqAAd2P8ArD', 'duration_ms': 267400, 'time_signature': 4}]
